# 1. Import libraries

In [1]:
#----------------------------Reproducible----------------------------------------------------------------------------------------
import numpy as np
import random as rn
import os

seed=0
os.environ['PYTHONHASHSEED'] = str(seed)

np.random.seed(seed)
rn.seed(seed)

#----------------------------Reproducible----------------------------------------------------------------------------------------

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

#--------------------------------------------------------------------------------------------------------------------------------
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
matplotlib.style.use('ggplot')

import random
import scipy.sparse as sparse
import scipy.io

from keras.utils import to_categorical
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import scipy.io
from skfeature.function.similarity_based import SPEC

import time
import pandas as pd

In [2]:
#--------------------------------------------------------------------------------------------------------------------------------
def ETree(p_train_feature,p_train_label,p_test_feature,p_test_label,p_seed):
    clf = ExtraTreesClassifier(n_estimators=50, random_state=p_seed)
    
    # Training
    clf.fit(p_train_feature, p_train_label)
    
    # Training accuracy
    print('Training accuracy：',clf.score(p_train_feature, np.array(p_train_label)))
    print('Training accuracy：',accuracy_score(np.array(p_train_label),clf.predict(p_train_feature)))
    #print('Training accuracy：',np.sum(clf.predict(p_train_feature)==np.array(p_train_label))/p_train_label.shape[0])

    # Testing accuracy
    print('Testing accuracy：',clf.score(p_test_feature, np.array(p_test_label)))
    print('Testing accuracy：',accuracy_score(np.array(p_test_label),clf.predict(p_test_feature)))
    #print('Testing accuracy：',np.sum(clf.predict(p_test_feature)==np.array(p_test_label))/p_test_label.shape[0])

In [3]:
#--------------------------------------------------------------------------------------------------------------------------------
def write_to_csv(p_data,p_path):
    dataframe = pd.DataFrame(p_data)
    dataframe.to_csv(p_path, mode='a',header=False,index=False,sep=',')

# 2. Loading data

In [4]:
train_data_frame=np.array(pd.read_csv('./Dataset/Isolet/isolet1+2+3+4.data',header=None))
test_data_frame=np.array(pd.read_csv('./Dataset/Isolet/isolet5.data',header=None))

train_data_arr=(train_data_frame[:,0:617]).copy()
train_label_arr=((train_data_frame[:,617]).copy()-1)
test_data_arr=(test_data_frame[:,0:617]).copy()
test_label_arr=((test_data_frame[:,617]).copy()-1)

Data=MinMaxScaler(feature_range=(0,1)).fit_transform(np.r_[train_data_arr,test_data_arr])

C_train_x=Data[:len(train_data_arr)]
C_test_x=Data[len(train_data_arr):]
C_train_y=train_label_arr#to_categorical(train_label_arr)
C_test_y=test_label_arr#to_categorical(test_label_arr)

In [5]:
x_train,x_validate,y_train_onehot,y_validate_onehot= train_test_split(C_train_x,C_train_y,test_size=0.1,random_state=seed)
x_test=C_test_x
y_test_onehot=C_test_y

print('Shape of x_train: ' + str(x_train.shape)) 
print('Shape of x_validate: ' + str(x_validate.shape)) 
print('Shape of x_test: ' + str(x_test.shape))
print('Shape of y_train: ' + str(y_train_onehot.shape))
print('Shape of y_validate: ' + str(y_validate_onehot.shape))
print('Shape of y_test: ' + str(y_test_onehot.shape))

print('Shape of C_train_x: ' + str(C_train_x.shape)) 
print('Shape of C_train_y: ' + str(C_train_y.shape)) 
print('Shape of C_test_x: ' + str(C_test_x.shape)) 
print('Shape of C_test_y: ' + str(C_test_y.shape)) 

Shape of x_train: (5614, 617)
Shape of x_validate: (624, 617)
Shape of x_test: (1559, 617)
Shape of y_train: (5614,)
Shape of y_validate: (624,)
Shape of y_test: (1559,)
Shape of C_train_x: (6238, 617)
Shape of C_train_y: (6238,)
Shape of C_test_x: (1559, 617)
Shape of C_test_y: (1559,)


In [6]:
key_feture_number=50

# 3. Classifying 1

### Extra Trees

In [7]:
train_feature=C_train_x
train_label=C_train_y
test_feature=C_test_x
test_label=C_test_y

print('Shape of train_feature: ' + str(train_feature.shape)) 
print('Shape of train_label: ' + str(train_label.shape)) 
print('Shape of test_feature: ' + str(test_feature.shape)) 
print('Shape of test_label: ' + str(test_label.shape)) 

p_seed=seed
ETree(train_feature,train_label,test_feature,test_label,p_seed)

Shape of train_feature: (6238, 617)
Shape of train_label: (6238,)
Shape of test_feature: (1559, 617)
Shape of test_label: (1559,)
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.9448364336112893
Testing accuracy： 0.9448364336112893


# 4. Model

In [8]:
start = time.clock()

# construct affinity matrix
kwargs = {'style': 0}

# obtain the scores of features, and sort the feature scores in an ascending order according to the feature scores
train_score = SPEC.spec(train_feature, **kwargs)

train_idx = SPEC.feature_ranking(train_score, **kwargs)

# obtain the dataset on the selected features
train_selected_x = train_feature[:, train_idx[0:key_feture_number]]
print("train_selected_x",train_selected_x.shape)


# obtain the scores of features, and sort the feature scores in an ascending order according to the feature scores
test_score = SPEC.spec(test_feature, **kwargs)

test_idx = SPEC.feature_ranking(test_score, **kwargs)

# obtain the dataset on the selected features
test_selected_x = test_feature[:, test_idx[0:key_feture_number]]
print("test_selected_x",test_selected_x.shape)


time_cost=time.clock() - start

write_to_csv(np.array([time_cost]),"./log/SPEC_time"+str(key_feture_number)+".csv")

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


train_selected_x (6238, 50)
test_selected_x (1559, 50)


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [9]:
C_train_selected_x=train_selected_x
C_test_selected_x=test_selected_x
C_train_selected_y=C_train_y
C_test_selected_y=C_test_y


print('Shape of C_train_selected_x: ' + str(C_train_selected_x.shape)) 
print('Shape of C_test_selected_x: ' + str(C_test_selected_x.shape)) 
print('Shape of C_train_selected_y: ' + str(C_train_selected_y.shape)) 
print('Shape of C_test_selected_y: ' + str(C_test_selected_y.shape)) 

Shape of C_train_selected_x: (6238, 50)
Shape of C_test_selected_x: (1559, 50)
Shape of C_train_selected_y: (6238,)
Shape of C_test_selected_y: (1559,)


# 5. Classifying 2

### Extra Trees

In [10]:
train_feature=C_train_selected_x
train_label=C_train_y

test_feature=C_test_selected_x
test_label=C_test_y

print('Shape of train_feature: ' + str(train_feature.shape)) 
print('Shape of train_label: ' + str(train_label.shape)) 
print('Shape of test_feature: ' + str(test_feature.shape)) 
print('Shape of test_label: ' + str(test_label.shape)) 

p_seed=seed
ETree(train_feature,train_label,test_feature,test_label,p_seed)

Shape of train_feature: (6238, 50)
Shape of train_label: (6238,)
Shape of test_feature: (1559, 50)
Shape of test_label: (1559,)
Training accuracy： 1.0
Training accuracy： 1.0
Testing accuracy： 0.05772931366260423
Testing accuracy： 0.05772931366260423


# 6. Reconstruction loss

In [11]:
from sklearn.linear_model import LinearRegression

def mse_check(train, test):
    LR = LinearRegression(n_jobs = -1)
    LR.fit(train[0], train[1])
    MSELR = ((LR.predict(test[0]) - test[1]) ** 2).mean()
    return MSELR

In [12]:
train_feature_tuple=(C_train_selected_x,C_train_x)
test_feature_tuple=(C_test_selected_x,C_test_x)

reconstruction_loss=mse_check(train_feature_tuple, test_feature_tuple)
print(reconstruction_loss)

0.11904186924248086
